In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D, Conv1D, LSTM
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from tensorflow.keras.utils import to_categorical 
import collections

import time

In [2]:
df = pd.read_csv('movieCorpus.csv')

In [3]:
df

,Unnamed: 0,movie_name,genre,original_language,popularity,release_date,vote_average,revenue,runtime,author,...,link_id,utc_datetime,body,Annotator1_Subjective,Annotator1_Polarity,Annotator2_Subjective,Annotator2_Polarity,Test,Auto_labeller_eval_subj,Auto_labeller_eval_pol
0,0,The Wolf of Snow Hollow,"Horror, Thriller",en,61.224,10/23/2020,6.1,239828,84,languinomaedtro,...,t3_vcginw,2022-06-15T00:00:00Z,The Wolf of snow hollow,None,None,None,None,0,OBJECTIVE,None
1,1,Borat Subsequent Moviefilm,Comedy,en,26.738,10/23/2020,6.5,0,96,extremelyhonesthou,...,t3_mykx8t,2021-04-26T00:00:00Z,What are we? Some kind of Borat Subsequent Mov...,None,None,None,None,0,OBJECTIVE,None
2,2,Level One,Adventure,en,17.256,1/20/2021,6.3,0,23,throwaway112112312,...,t3_hszwbj,2020-07-18T00:00:00Z,I love Yojimbo but Harakiri is on another leve...,None,None,None,None,0,OBJECTIVE,None
3,3,Gretel & Hansel,"Horror, Fantasy, Thriller",en,22.828,1/30/2020,6.1,22059211,87,mmmyesplease---,...,t3_yg0oab,2022-10-29T00:00:00Z,They were just treasures we found at libraries...,None,None,None,None,0,OBJECTIVE,None
4,4,47 Meters Down: Uncaged,"Horror, Drama, Adventure",en,16.081,8/15/2019,5.7,0,90,had0ukenn,...,t3_pehuql,2021-08-30T00:00:00Z,This is directed by Johannes Roberts who direc...,None,None,None,None,0,OBJECTIVE,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21649,21649,The Beta Test,"Comedy, Mystery, Thriller, Horror",en,30.294,10/15/2021,5.8,0,93,SpecialistTax6798,...,t3_xla7w0,2022-09-22T00:00:00Z,Recently I quite enjoyed The Beta Test.,SUBJECTIVE,POSITIVE,OBJECTIVE,None,1,OBJECTIVE,None
21650,21650,Promising Young Woman,"Thriller, Crime, Drama",en,36.116,12/13/2020,7.6,13868965,114,Naweezy,...,t3_10k68y9,2023-01-24T00:00:00Z,Carey Mulligan shouldve have won that yr for P...,SUBJECTIVE,POSITIVE,OBJECTIVE,None,1,OBJECTIVE,None
21651,21651,The Green Knight,"Adventure, Drama, Fantasy",en,197.458,7/29/2021,6.7,18192220,130,Naweezy,...,t3_104tjjk,2023-01-06T00:00:00Z,His wife Alicia Vikander should be in more mov...,SUBJECTIVE,NEUTRAL,OBJECTIVE,None,1,OBJECTIVE,None
21652,21652,Those Who Wish Me Dead,Thriller,en,130.330,2021-05-05T00:00:00Z,7.0,23384502,100,thejugglar,...,t3_vm8u6d,2022-06-28T00:00:00Z,"He played an ex military hit man in ""Those who...",SUBJECTIVE,NEUTRAL,OBJECTIVE,None,1,OBJECTIVE,None


In [4]:
# Rename columns
new_col_name = {
                "Auto_labeller_eval_subj": "subj",
                "Auto_labeller_eval_pol": "pol",
                "Test":"test"
               }
df = df.rename(columns=new_col_name)
df.head()

,Unnamed: 0,movie_name,genre,original_language,popularity,release_date,vote_average,revenue,runtime,author,...,link_id,utc_datetime,body,Annotator1_Subjective,Annotator1_Polarity,Annotator2_Subjective,Annotator2_Polarity,test,subj,pol
0,0,The Wolf of Snow Hollow,"Horror, Thriller",en,61.224,10/23/2020,6.1,239828,84,languinomaedtro,...,t3_vcginw,2022-06-15T00:00:00Z,The Wolf of snow hollow,None,None,None,None,0,OBJECTIVE,None
1,1,Borat Subsequent Moviefilm,Comedy,en,26.738,10/23/2020,6.5,0,96,extremelyhonesthou,...,t3_mykx8t,2021-04-26T00:00:00Z,What are we? Some kind of Borat Subsequent Mov...,None,None,None,None,0,OBJECTIVE,None
2,2,Level One,Adventure,en,17.256,1/20/2021,6.3,0,23,throwaway112112312,...,t3_hszwbj,2020-07-18T00:00:00Z,I love Yojimbo but Harakiri is on another leve...,None,None,None,None,0,OBJECTIVE,None
3,3,Gretel & Hansel,"Horror, Fantasy, Thriller",en,22.828,1/30/2020,6.1,22059211,87,mmmyesplease---,...,t3_yg0oab,2022-10-29T00:00:00Z,They were just treasures we found at libraries...,None,None,None,None,0,OBJECTIVE,None
4,4,47 Meters Down: Uncaged,"Horror, Drama, Adventure",en,16.081,8/15/2019,5.7,0,90,had0ukenn,...,t3_pehuql,2021-08-30T00:00:00Z,This is directed by Johannes Roberts who direc...,None,None,None,None,0,OBJECTIVE,None


In [5]:
#Keep only relevant columns
keep_col = ['body', 'test', 'subj', 'pol']
df = df[keep_col]

df.head()

,body,test,subj,pol
0,The Wolf of snow hollow,0,OBJECTIVE,None
1,What are we? Some kind of Borat Subsequent Mov...,0,OBJECTIVE,None
2,I love Yojimbo but Harakiri is on another leve...,0,OBJECTIVE,None
3,They were just treasures we found at libraries...,0,OBJECTIVE,None
4,This is directed by Johannes Roberts who direc...,0,OBJECTIVE,None


In [6]:
train = df.loc[df['test'] == 0]
test = df.loc[df['test'] == 1]

In [7]:
train.shape

(19489, 4)

In [8]:
test.shape

(2165, 4)

# Data Processing

In [9]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [10]:
# remove any sort of remaining tags
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [11]:
train_X_subj = []
sentences = list(train['body'])
for sen in sentences:
    train_X_subj.append(preprocess_text(sen))

test_X_subj = []
sentences = list(test['body'])
for sen in sentences:
    test_X_subj.append(preprocess_text(sen))
    
train_y_subj = train['subj']
test_y_subj = test['subj']

In [12]:
def change_subj_labels(data):
    #classification
    #group = {-1:0, 0:1, 1:2}
    group_subj = {"OBJECTIVE":0, "SUBJECTIVE":1}
    data = data.apply(lambda x: group_subj[x])
    return data
train_y_subj = change_subj_labels(train_y_subj).values
test_y_subj = change_subj_labels(test_y_subj).values

In [13]:
train_pol = train.loc[train['subj'] == "SUBJECTIVE"]
test_pol = test.loc[test['subj'] == "SUBJECTIVE"]

In [14]:
train_X_pol = []
sentences = list(train_pol['body'])
for sen in sentences:
    train_X_pol.append(preprocess_text(sen))

test_X_pol = []
sentences = list(test_pol['body'])
for sen in sentences:
    test_X_pol.append(preprocess_text(sen))
    
train_y_pol = train_pol['pol']
test_y_pol = test_pol['pol']

In [15]:
def change_pol_labels(data):
    #classification
    #group = {-1:0, 0:1, 1:2}
    group_pol = {"NEGATIVE":0, "NEUTRAL":1, "POSITIVE":2}
    data = data.apply(lambda x: group_pol[x])
    return data
train_y_pol = change_pol_labels(train_y_pol).values
test_y_pol = change_pol_labels(test_y_pol).values

# Word embedding with GloVe from Stanford 

In [16]:
tokenizer = Tokenizer(num_words=1000) # Total number of words to be considered in the tokenizer, changeable
tokenizer.fit_on_texts(train_X_subj)

train_X_subj = tokenizer.texts_to_sequences(train_X_subj)
test_X_subj = tokenizer.texts_to_sequences(test_X_subj)

In [17]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100 # Max length to consider in a single review, changeable

train_X_subj = pad_sequences(train_X_subj, padding='post', maxlen=maxlen)
test_X_subj = pad_sequences(test_X_subj, padding='post', maxlen=maxlen)

In [18]:
import os
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

# Get the current working directory
cwd = os.getcwd()

# Define the relative path to the file
file_path = "glove.6B.100d.txt"

# Join the paths to create the full file path
full_path = os.path.join(cwd, file_path)

# Open the file using the full path
glove_file = open(full_path, encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [19]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [20]:
tokenizer1 = Tokenizer(num_words=1000) # Total number of words to be considered in the tokenizer, changeable
tokenizer1.fit_on_texts(train_X_pol)
train_X_pol = tokenizer1.texts_to_sequences(train_X_pol)
test_X_pol = tokenizer1.texts_to_sequences(test_X_pol)

vocab_size1 = len(tokenizer1.word_index) + 1

maxlen = 100 # Max length to consider in a single review, changeable

train_X_pol = pad_sequences(train_X_pol, padding='post', maxlen=maxlen)
test_X_pol = pad_sequences(test_X_pol, padding='post', maxlen=maxlen)

embedding_matrix1 = zeros((vocab_size1, 100))
for word, index in tokenizer1.word_index.items():
    embedding_vector1 = embeddings_dictionary.get(word)
    if embedding_vector1 is not None:
        embedding_matrix1[index] = embedding_vector1

# Testing using Simple Neural Network

In [21]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

def accuracy_pol(preds, labels):
    label_dict_inverse = {0: "Negative", 2: "Positive", 1: "Neutral"}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')
        
def accuracy_subj(preds, labels):
    label_dict_inverse = {0: "Objective", 1: "Subjective"}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [22]:
train_y_subj = to_categorical(train_y_subj)
test_y_subj = to_categorical(test_y_subj)

In [23]:
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)

model.add(Flatten())
model.add(Dense(2, activation='softmax'))

In [24]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          2890200   
_________________________________________________________________
flatten (Flatten)            (None, 10000)             0         
_________________________________________________________________
dense (Dense)                (None, 2)                 20002     
Total params: 2,910,202
Trainable params: 20,002
Non-trainable params: 2,890,200
_________________________________________________________________
None


In [25]:
history = model.fit(train_X_subj, train_y_subj, batch_size=248, epochs=20, verbose=1, validation_split=0.2)

Epoch 1/20
63/63 [==============================] - 1s 11ms/step - loss: 0.7119 - acc: 0.5787 - val_loss: 0.4927 - val_acc: 0.8004
Epoch 2/20
63/63 [==============================] - 1s 9ms/step - loss: 0.6138 - acc: 0.6715 - val_loss: 0.4948 - val_acc: 0.7753
Epoch 3/20
63/63 [==============================] - 1s 8ms/step - loss: 0.5813 - acc: 0.7001 - val_loss: 0.5987 - val_acc: 0.6609
Epoch 4/20
63/63 [==============================] - 1s 9ms/step - loss: 0.5555 - acc: 0.7207 - val_loss: 0.5173 - val_acc: 0.7512
Epoch 5/20
63/63 [==============================] - 1s 8ms/step - loss: 0.5355 - acc: 0.7359 - val_loss: 0.5907 - val_acc: 0.6868
Epoch 6/20
63/63 [==============================] - 0s 8ms/step - loss: 0.5187 - acc: 0.7484 - val_loss: 0.5425 - val_acc: 0.7237
Epoch 7/20
63/63 [==============================] - 0s 8ms/step - loss: 0.5061 - acc: 0.7592 - val_loss: 0.5593 - val_acc: 0.7142
Epoch 8/20
63/63 [==============================] - 1s 9ms/step - loss: 0.4984 - acc: 0.7

In [26]:
# score = model.evaluate(test_X_subj, test_y_subj, verbose=1)

In [27]:
# print("Test Score:", score[0])
# print("Test Accuracy:", score[1])

In [28]:
start = time.time()

subj_pred = model.predict(test_X_subj)

end = time.time()
print("Time taken to predict: ", end - start)

Time taken to predict:  0.2629380226135254


In [29]:
labels_subj = np.argmax(test_y_subj, axis=1).flatten()
preds_subj = np.argmax(subj_pred, axis=1).flatten()

In [30]:
counter = collections.Counter(labels_subj)
counter

Counter({1: 1518, 0: 647})

In [31]:
counter = collections.Counter(preds_subj)
counter

Counter({0: 704, 1: 1461})

In [32]:
accuracy_score(labels_subj, preds_subj)

0.7066974595842956

In [33]:
print(classification_report(labels_subj, preds_subj))

              precision    recall  f1-score   support

           0       0.51      0.55      0.53       647
           1       0.80      0.77      0.79      1518

    accuracy                           0.71      2165
   macro avg       0.66      0.66      0.66      2165
weighted avg       0.71      0.71      0.71      2165



In [34]:
# confusion_matrix(labels_subj, subj_pred)

In [35]:
f1_score(labels_subj, preds_subj, average = 'weighted')

0.7100788026273954

In [36]:
accuracy_subj(subj_pred,labels_subj)

Class: Objective
Accuracy:358/647

Class: Subjective
Accuracy:1172/1518



In [37]:
model1 = Sequential()
embedding_layer1 = Embedding(vocab_size1, 100, weights=[embedding_matrix1], input_length=maxlen , trainable=False)
model1.add(embedding_layer1)

model1.add(Flatten())
model1.add(Dense(3, activation='softmax'))

In [38]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          2340400   
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 30003     
Total params: 2,370,403
Trainable params: 30,003
Non-trainable params: 2,340,400
_________________________________________________________________
None


In [39]:
train_y_pol = to_categorical(train_y_pol)
test_y_pol = to_categorical(test_y_pol)

In [40]:
history1 = model1.fit(train_X_pol, train_y_pol, batch_size=248, epochs=20, verbose=1, validation_split=0.2)

Epoch 1/20
40/40 [==============================] - 1s 14ms/step - loss: 1.0689 - acc: 0.4572 - val_loss: 1.0336 - val_acc: 0.5004
Epoch 2/20
40/40 [==============================] - 0s 9ms/step - loss: 0.9213 - acc: 0.5659 - val_loss: 0.9950 - val_acc: 0.5054
Epoch 3/20
40/40 [==============================] - 0s 9ms/step - loss: 0.8448 - acc: 0.6262 - val_loss: 0.9844 - val_acc: 0.5326
Epoch 4/20
40/40 [==============================] - 0s 10ms/step - loss: 0.7971 - acc: 0.6569 - val_loss: 0.9908 - val_acc: 0.5268
Epoch 5/20
40/40 [==============================] - 0s 9ms/step - loss: 0.7594 - acc: 0.6823 - val_loss: 0.9786 - val_acc: 0.5305
Epoch 6/20
40/40 [==============================] - 0s 9ms/step - loss: 0.7268 - acc: 0.7033 - val_loss: 0.9815 - val_acc: 0.5375
Epoch 7/20
40/40 [==============================] - 0s 9ms/step - loss: 0.6991 - acc: 0.7143 - val_loss: 0.9828 - val_acc: 0.5326
Epoch 8/20
40/40 [==============================] - 0s 8ms/step - loss: 0.6771 - acc: 0.

In [41]:
score1 = model1.evaluate(test_X_pol, test_y_pol, verbose=1)

48/48 [==============================] - 0s 2ms/step - loss: 1.0210 - acc: 0.5982


In [42]:
start = time.time()

pol_pred = model1.predict(test_X_pol)

end = time.time()
print("Time taken to predict: ", end - start)

Time taken to predict:  0.1412665843963623


In [43]:
labels_pol = np.argmax(test_y_pol, axis=1).flatten()
preds_pol = np.argmax(pol_pred, axis=1).flatten()

In [44]:
counter = collections.Counter(labels_pol)
counter

Counter({2: 710, 1: 315, 0: 493})

In [45]:
counter = collections.Counter(preds_pol)
counter

Counter({2: 578, 0: 425, 1: 515})

In [46]:
accuracy_score(labels_pol, preds_pol)

0.5981554677206851

In [47]:
print(classification_report(labels_pol, preds_pol))

              precision    recall  f1-score   support

           0       0.67      0.58      0.62       493
           1       0.34      0.55      0.42       315
           2       0.78      0.63      0.70       710

    accuracy                           0.60      1518
   macro avg       0.60      0.59      0.58      1518
weighted avg       0.65      0.60      0.61      1518



In [48]:
# confusion_matrix(labels_pol, preds_pol)

In [49]:
f1_score(labels_pol, preds_pol, average = 'weighted')

0.6149630870537182

In [50]:
accuracy_pol(pol_pred,labels_pol)

Class: Negative
Accuracy:286/493

Class: Neutral
Accuracy:173/315

Class: Positive
Accuracy:449/710

